## Extract the infomation in the dataframe

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import lxml
import numpy as np

# handle website
year = np.arange(103 ,112, 1)
month = np.arange(1,13,1)
year_list = []

for page in year:
    page = "https://mops.twse.com.tw/nas/t21/sii/t21sc03_"+str(page)+"_1_0.html"
    year_list.append(page)
    

final_year_month = []
for i in year_list:
    for m in month:
        j = str(i[:49]) + str(m) + str(i[50:])
        final_year_month.append(j)

# Create a sequence of list 
def createList(r1, r2):
    return [item for item in range(r1, r2+1)]

# For extracting dataframe
r1, r2 = 2, 58
numbers = createList(r1, r2)
even = [i for i in numbers if i%2 == 0]

# To extract 產業別
a1, a2 = 1, 57
numberss = createList(a1, a2)
odd = [i for i in numberss if i%2 != 0]
    

result = pd.DataFrame()
result_last = pd.DataFrame()

In [2]:
#final_year_month = final_year_month[:-5]
final_year_month = final_year_month[:-7]

In [3]:
final_year_month

['https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_1_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_2_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_3_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_4_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_5_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_6_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_7_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_8_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_9_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_10_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_11_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_103_12_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_104_1_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_104_2_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_104_3_0.html',
 'https://mops.twse.com.tw/nas/t21/sii/t21sc03_104_4_0.html',
 'htt

In [4]:
new_column = []
for num in range(len(final_year_month)):
    if(final_year_month[num][45:51][-1] == '_'):
        new_column.append(final_year_month[num][45:50])
    else:
        new_column.append(final_year_month[num][45:51])

In [5]:
iterator = 0

for i in range(len(final_year_month)):
    new_df = pd.DataFrame()
    final_df = pd.DataFrame()
    industry = pd.DataFrame()


    print('This is one single month: ', i)
    # Define new empty dataframe

    column_name = str(iterator)
    
    url = final_year_month[i]
    
    res = requests.get(url) 
    chineese_encoding = res.apparent_encoding
    res.encoding = chineese_encoding
    soup = BeautifulSoup(res.text,"html.parser") 

    # Append every dataframe from the website
    for kkk in even:
        data = soup.select('table')[kkk]
        #print(data)

        df = pd.read_html(data.prettify())
        dfs = df[0]
        new_df = new_df.append(dfs)
    
    new_df.columns = new_df.columns.get_level_values(1)
    final = new_df.loc[new_df["公司名稱"] == "合計"].reset_index(drop= True)
    final_df["產業別"]= final["公司名稱"]
    #final_df.loc[:, column_name] = final["當月營收"]
    final_df["當月營收"]= final["當月營收"]
    final_df = final_df.rename(columns = {'當月營收':new_column[i]})


    for jjj in odd:
        data = soup.select('table')[jjj]
        df = pd.read_html(data.prettify())
        dfs = df[0]

        new_list = []
        real_list = []
        for mmm in dfs.columns.values.tolist():
            new_list.append(mmm)
            break

        for ggg in new_list:
            real_list.append(ggg[4:])
            break

        industry = industry.append(real_list)
    
    industry = industry.reset_index(drop= True)
    industry.columns = ['公司名稱']
    final_df["產業別"] = industry["公司名稱"]
    #print(final_df)
    result = result.append(final_df)
    iterator += 1
    

This is one single month:  0
This is one single month:  1
This is one single month:  2
This is one single month:  3
This is one single month:  4
This is one single month:  5
This is one single month:  6
This is one single month:  7
This is one single month:  8
This is one single month:  9
This is one single month:  10
This is one single month:  11
This is one single month:  12
This is one single month:  13
This is one single month:  14
This is one single month:  15
This is one single month:  16
This is one single month:  17
This is one single month:  18
This is one single month:  19
This is one single month:  20
This is one single month:  21
This is one single month:  22
This is one single month:  23
This is one single month:  24
This is one single month:  25
This is one single month:  26
This is one single month:  27
This is one single month:  28
This is one single month:  29
This is one single month:  30
This is one single month:  31
This is one single month:  32
This is one single m

In [6]:
result = result.apply(lambda x: pd.Series(x.dropna().values))

In [7]:
result = result.dropna()
result

,產業別,103_1,103_2,103_3,103_4,103_5,103_6,103_7,103_8,103_9,...,110_8,110_9,110_10,110_11,110_12,111_1,111_2,111_3,111_4,111_5
0,水泥工業,17930828.0,10339592.0,20025052.0,20312201.0,20096893.0,18306415.0,18646873.0,18144334.0,18068218.0,...,17705599.0,18945697.0,20860429.0,20738870.0,22248155.0,16747415.0,10528814.0,19959347.0,18805611.0,17510737.0
1,食品工業,66303579.0,48920151.0,56785298.0,56500771.0,57371556.0,57783831.0,61554613.0,61749963.0,59766812.0,...,67703366.0,68724535.0,64788047.0,67465050.0,66723397.0,78119692.0,55064385.0,69128233.0,69165830.0,69269538.0
2,塑膠工業,102958314.0,91550423.0,103472527.0,108894445.0,108940412.0,107647488.0,109940468.0,104675842.0,97873277.0,...,118890219.0,114887674.0,121712690.0,120541242.0,117161182.0,114504785.0,100244824.0,127628191.0,116934876.0,119555369.0
3,紡織纖維,44896366.0,42325884.0,45405806.0,44994932.0,45406818.0,45527245.0,46031050.0,46618904.0,46835431.0,...,46150313.0,45617996.0,47540360.0,50464017.0,52867443.0,49089066.0,45140549.0,55349516.0,51185707.0,53245342.0
4,電機機械,26701012.0,24520194.0,30600741.0,27261511.0,28082317.0,31537874.0,28407800.0,27232904.0,29369831.0,...,28575072.0,28253143.0,27613310.0,29465418.0,32536837.0,28777972.0,23100039.0,30698764.0,25850010.0,26966823.0
5,電器電纜,18914208.0,16946890.0,20318980.0,20864498.0,21874784.0,21323307.0,21791719.0,18908355.0,23836166.0,...,21250503.0,22559536.0,22026887.0,22820818.0,25525949.0,21654550.0,19243412.0,25076592.0,25213742.0,25773498.0
6,化學工業,22840857.0,18401507.0,22892239.0,23828606.0,22424452.0,21479985.0,21859149.0,22169345.0,22871597.0,...,24484785.0,24820126.0,25819662.0,26707991.0,27146707.0,27274895.0,22267129.0,29856859.0,27794511.0,27022621.0
7,生技醫療業,6237078.0,4977501.0,5903654.0,5972248.0,6513055.0,5821060.0,5800254.0,5881677.0,6542139.0,...,9959138.0,11383896.0,10969731.0,11297157.0,13020150.0,10449697.0,8929850.0,11331252.0,9696774.0,10405732.0
8,玻璃陶瓷,4164858.0,3671050.0,5090297.0,5272911.0,5429080.0,5224195.0,5266222.0,5159852.0,5236364.0,...,6375185.0,5818728.0,5691328.0,5794035.0,6123732.0,4785496.0,4741467.0,5084081.0,5085585.0,5351753.0
9,造紙工業,12469073.0,11236139.0,12367516.0,12976378.0,12954312.0,12650997.0,13118893.0,12912276.0,13214537.0,...,17191855.0,19109280.0,18940183.0,19686383.0,20209581.0,18697761.0,14689839.0,20392480.0,17603163.0,18617189.0


In [8]:
result.to_csv("monthly_revenue.csv")